In [76]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math



In [77]:
#Defines the initial mean and skew for a given depression level to be used at the start of a simulation
mean_interval,skew_interval,mean_overlap,skew_overlap = 0.034,0.14,0.034/4,0.035
def initial(dep,mean_interval,skew_interval,mean_overlap,skew_overlap):
    decimal = dep%1
    dep = int(dep)
    if dep>1 or dep<0:
        return 'error'

    
    if dep !=0 and dep!=0.9 and dep!=1:
        mn = 0.2 +mean_overlap-(dep*10)*mean_interval - (mean_interval+2*mean_overlap)*decimal
        sk = 0 +  skew_overlap-(dep*10)*skew_interval - (skew_interval+2*mean_overlap)*decimal
    
    elif dep==0.9:
        mn = 0.2 - 10*mean_interval+(mean_interval+mean_overlap)*(1-decimal) 
        sk = 0-10*skew_interval + (skew_interval+skew_overlap)*(1-decimal)
        
    elif dep==0:
        mn = 0.2 - (mean_interval+mean_overlap)*decimal
        sk = 0 - (mean_interval+mean_overlap)*decimal

    elif dep==1:
        mn = 0.2 - 10*mean_interval
        sk = 0-10*skew_interval
       
    return [mn,sk]

In [78]:
#Defines how the mean and skew will change during a decision

skew_change,mean_change = (skew_interval+2*skew_overlap)/10, (0.034+2*mean_overlap)/10 #Change in mean and skew during a decision
def deliberation_no(mean_in,skew_in):
    skew = skew_in-skew_change
    mean = mean_in - mean_change

    
    return [mean,skew]
    
def deliberation_yes(mean_in,skew_in):
    skew = skew_in + skew_change
    mean = mean_in + mean_change

    
    
    return [mean,skew]

In [79]:
#Energy used per deliberation
def en_delib(dep):
    return (0.1+dep*2.5)*0.69

In [80]:
#The process of making a decision to engage or not by sampling under the gaussian

#while loop ensures no value greater than 1 
def decision_process(mean,skew,dep):
  yes = no = 99
  while yes>1 or no>1:
    
    yes = stats.skewnorm.rvs(skew,mean,0.2)
    no = np.random.normal(0,0.1)
    
    
    flg=0
    
    
    
    val = abs(yes-no)
    
    en = en_delib(dep)/val
    
    
    if en >1.5*0.69:
      en = 1.5*0.69
    if yes>no:
      flg=1

    return [flg,val,en,yes,no]

In [81]:
#one entire decision
def one_decision(mn,sk,dep):
    
    rep = 0 #number of processes in the decision

    pos = 0 #value assigned to engaging, if it reaches pos_th the decision is engage
    neg = 0 #value of not engaging, if it reaches neg_th the decision is not engage
    en = 0  #value of energy used, if it reaches en_th the decision is not engage

    yes = 0 #total engage value sampled when it is greater than not engage, doesn't include the subtraction in val
    no = 0  #total not engage value sampled when it is greater than engage
    ys = 0  #total times engage was greater
    n = 0   #total times not engage was greater

    en_flg = 0 #indicates if en reached en_th
    want = 0 #if exhaustion is reached, but the subject wanted to engage, this has a value and is used in the learning

    mn_dec = mn #Can change during a decision
    sk_dec = sk #Can change during a decision
    

    while pos < 0.4 and neg < 0.7 and en<8:
        
        deliberation = decision_process(mn_dec,sk_dec,dep)

        if deliberation[0] == 1:   #engage
            pos_change = deliberation_yes(mn_dec,sk_dec)
            mn_dec = pos_change[0]
            sk_dec = pos_change[1]
            pos +=deliberation[1]
            yes+=deliberation[3]
            ys+=1 

        else: #disengage
            neg_change = deliberation_no(mn_dec,sk_dec)
            mn_dec = neg_change[0]
            sk_dec = neg_change[1]
            neg +=deliberation[1]
            no+=deliberation[4]
            n+=1

        en+=deliberation[2]
        rep+=1
    
    if en>8 and pos<0.4 and neg<0.7:
        en_flg = 1
        if pos>neg:
            want = yes/ys

    if pos<0.4: #stops division by zero
        if n==0:
            n = 1
            no += deliberation[4]
            
    
    
    return [1,en,yes/rep,rep,en_flg,want] if pos>0.4 else [0,en,no/n,rep,en_flg,want]

In [82]:
#one entire decision when CBTb is active, blocks any choseing to disengage, can ony engage or exhaust

def one_decision_CBT(mn,sk,dep,en,pos):
    
    rep = 0 #number of processes in the decision

    
    neg = 0 #value of not engaging, if it reaches neg_th the decision is not engage and a loop occurs
    

    yes = 0 #total engage value sampled when it is greater than not engage
   
    ys = 0  #total times engage was greater
    n = 0   #total times not engage was greater

    en_flg = 0 #indicates if en reached en_th
    want = 0 #if exhaustion is reached, but the subject wanted to engage, this has a value

    mn_dec = mn #Can change during a decision
    sk_dec = sk #Can change during a decision
   

    while pos < 0.4 and neg < 0.7 and en<8:
        
        deliberation = decision_process(mn_dec,sk_dec,dep)

        if deliberation[0] == 1:   #engage
            pos_change = deliberation_yes(mn_dec,sk_dec)
            mn_dec = pos_change[0]
            sk_dec = pos_change[1]
            pos +=deliberation[1]
            yes+=deliberation[3]
            ys+=1 

        else: #disengage
            neg_change = deliberation_no(mn_dec,sk_dec)
            mn_dec = neg_change[0]
            sk_dec = neg_change[1]
            neg +=deliberation[1]
            
            n+=1

        en+=deliberation[2]
        rep+=1
    
    if en>8 and pos<0.4 and neg<0.7:
        en_flg = 1
        if pos>neg and ys!=0:
            want = yes/ys
    
        
    return [1,en,yes,ys,rep,en_flg,want] if pos>0.4 else [0,en,yes,ys,rep,en_flg,want]

In [83]:
#Maximum daily energy use, might make constant
def day_en_th_fun(x):
    x = x*10
    return (-176/117*x**2+1058/117*x+100)*1.5/0.8*1.3

In [84]:
#Functions to manualy alter the course of the model
#day will be an array of days when alterations occur
#del_* will be the changes which occir on that day
#Sequential days will have the same del_*
#brakes in sequential days will have value -1, this moves on to the next del_* value

def force_mean(day:[],del_m1:[],d):
    if day:
        if d ==day[-1]:
            day.pop()
            return del_m1[-1]
            
        elif day[-1]==-1:
            del_m1.pop()
            day.pop()
        
    return 0 




def force_skew(day:[],del_sk:[],d):
    if day:
        if d ==day[-1]:
            day.pop()
            
            return del_sk[-1]
            
        elif day[-1]==-1:
            del_sk.pop()
            day.pop()
        
    return 0 



def force_actual(day:[],actual_pos:[],d): #forces a  value for actual positivity of event after engaging

    if day:
        if d ==day[-1]:
            day.pop()
            return actual_pos[-1]
            
        elif day[-1]==-1:
            actual_pos.pop()
            day.pop()
        
    return -99



def force_energy(day:[],d):  #forces high energy usage to force not engage without affecting distributions
    if day:
        if d ==day[-1]:
            day.pop()
            return 1000
        
    return 0 



def force_ap(day:[],del_ap:[],d):

    if day:
        if d ==day[-1]:
            day.pop()
            return del_ap[-1]
            
        elif day[-1]==-1:
            del_ap.pop()
            day.pop()
        
    return 0 

def force_mood(day:[],mood:[],d): #Force a maximum mood value for a period of days
    if day:
        if d ==day[-1]:
            day.pop()
            return mood[-1]
            
        elif day[-1]==-1:
            mood.pop()
            day.pop()
        
    return 1



def force_disengage(day:[],d): #Force disengagement for a period of days, set values (deliberation) as NaN and use numpy nanmean, force disengage or loop until disengage is chosen
    if day:
        if d ==day[-1]:
            day.pop()
            return 1
        
    return 0 





In [85]:
#f is in the format [day,duration,val,day,duration,val...]
#day the tool is active, number of days the tool is active, value which the tool changes by
# returns an array of active days and their active values

def create_lists(f):
    if f is None:
        return [-2],[-2]
    days = []
    dels = []
    
    for i in range(int(len(f)/3)):
        start = f[i*3]
        duration = f[i*3+1]+1
        val=f[i*3+2]
        dur = np.arange(start,start+duration,1)
        days.extend(dur)
        days.append(-1)
        dels.append(val)
    days.reverse()
    dels.reverse()
    return days, dels


def create_lists_en(f):
    if f is None:
        return [-2]
    days = []
    for i in range(int(len(f)/2)):
        start = f[i*2]
        duration = f[i*2+1]+1
        dur = np.arange(start,start+duration,1)
        days.extend(dur)
    days.reverse()
    return days



In [86]:
def CBTb(day:[],d): # return1 if active 0 if not
    if day:
        if d ==day[-1]:
            day.pop()
            return 1
        elif day[-1]==-1:
                day.pop()
    return 0

def CBTc(day:[] ,d , s): # increases skew when active and not above max, otherwise 0
    
    if day:
        if d ==day[-1]:
            day.pop()
            if s>=1.3:
                return s
            
                
            else:
                 val = np.random.normal(75,75*2/5)*0.0005
                 
            return s+val
            
        elif day[-1]==-1:
            del_sk.pop()
            day.pop()
    return s

In [87]:

def mean(md,m1):
    mu = m1-md*0.034*10
    return mu

def reward_sens(dep):
    
    return (1-dep/2)

def param_learn(mood,m1,val,flg,flg_con,skew,d,day_actual,actual_pos,dep,exhaust,want): 
    
    del_sk = 0
    m1_new = m1

    actual = force_actual(day_actual,actual_pos,d) #Forces a value for the actual outcome
    if actual == -99: #vaue when we do not force an actual outcome
        actual = np.random.normal(0.2,0.2)*reward_sens(dep)
    

    if flg==1: #engage
        dif = (actual-val) #actual*reward sensitivity - predicted
        
        m1_new = m1+0.0005*dif
        
        
            
        del_mood = actual
    elif exhaust ==1:
        if want!=0:
            del_mood = np.random.normal(0,0.1) - want
        mn = mean((1-mood),m1)
        m, vr, sk, kurt = stats.skewnorm.stats(skew,mn,0.2, moments='mvsk')
        
        del_m1 = m1 - (m*vr**2)/(0.1**2+vr**2) # multiplication of current and not engage gaussians, without updating var, modified Bayesian Inference
        m1_new = m1 - del_m1*0.0005*0.1
        del_mood = np.random.normal(0,0.1/2)
       

            

    elif flg==0:
        mn = mean((1-mood),m1)
        m, vr, sk, kurt = stats.skewnorm.stats(skew,mn,0.2, moments='mvsk')
        
        del_m1 = m1 - (m*vr**2)/(0.1**2+vr**2) # multiplication of current and not engage gaussians, without updating var, modified Bayesian Inference
        m1_new = m1 - del_m1*0.0005
        del_mood = np.random.normal(0,0.1)

       
        
    #If two consecutive engages apathy improves (towards 0), if currently not engaging apathy gets worse, else no change
    del_ap = -0.0005*40*2
    if flg_con ==0:
        del_ap*=-1 
        if flg == 1:
            del_ap = 0
    

    
    return(m1_new,del_sk,del_ap,del_mood)

def learn_sk(engage,sk):
    if sk<0.2 and sk>-0.9:
        del_sk = (engage-5)/10*0.0005*40*3
    elif sk>0.2:
        del_sk = -0.01
    else:
        del_sk = 0.01
    
    return del_sk


    





In [88]:
#All decisions across d days
#f_ are functions that force values in an analysis, uses lists which are created here
def day_decisions(dep , days,f_mn,f_sk,f_ap,f_ac,f_en,f_CBTc,f_ILM,f_ID,f_CBTb):
  

    
    
    ap = 10*dep/0.6 # defines an initial apathy score that reflects the ap needed to give a the inital dep
    
    #All lists generated for perturbing the model, gives the days to perturb and any perturbing values
    day_m1,del_m1 = create_lists(f_mn)
    day_sk,del_sk = create_lists(f_sk)
    day_ap,del_ap = create_lists(f_ap)
    day_actual,actual_pos = create_lists(f_ac)
    day_energy = create_lists_en(f_en)
    day_CBTc = create_lists_en(f_CBTc)
    day_mood,mood_cap = create_lists(f_ILM)
    day_disengage = create_lists_en(f_ID)
    day_CBTb = create_lists_en(f_CBTb)

    start = initial(dep,mean_interval,skew_interval,mean_overlap,skew_overlap)

    sk = start[1]
    
    
    m1=0.2 #allows m1 to change
    
    
    flg = 1 #flag if the current decisions is engage, also used to indicate if the previous decision engaged
    
    
    #evolution over all days
    ev_mn=[]
    ev_sk=[]
    ev_ap =[]
    ev_pos_dec = []
    ev_neg_dec = []
    ev_postoneg = []
    ev_rep = []
    ev_exhaust = []
    ev_day_exhaust = []
    ev_mood = []
    ev_dep = []

    

    for d in range(days):

        mood = -1
        while mood<0 or mood >1:
            mood =  np.random.normal(0.9-dep*0.8,0.1)
        md_cap = force_mood(day_mood,mood_cap,d) #forcing a period of low mood
        if mood > md_cap: #Cap is 1 when we dont force mood
                mood = md_cap
        
       

        
        sk = CBTc(day_CBTc, d, sk) #ADM changing the skew, the learning process will deal with this once ADM is stopped
        
        day_en = 0 #energy used in  a day
        dec = 0 # max of 10 decisions in a day
        yes = 0  #total engage decisions in a day
        no = 0 #total number of no's in a day
        en_flg = 0 #total number of exhausts from one decision in a day
        day_exhaust = 0 #decisions after which a complete exhaustion occurs

        day_en_th = day_en_th_fun(dep) 
        

        
        sk+=force_skew(day_sk,del_sk,d) #can force a change in skew
        ap+=force_ap(day_ap,del_ap,d) #can force a change in ap
        m1+=force_mean(day_m1,del_m1,d) # can fore a change in m1 hence mean
        disengage = force_disengage(day_disengage,d) #Force disengagement for a period
        cbtb = CBTb(day_CBTb,d)

        while dec<10 and day_en<day_en_th:
            
            mn = mean((1-mood),m1) # mean of engage depends on mood
            if cbtb ==1: #CBTb is active, cannot actively disengage only engage or exhaust
                
                en,pos,outcome,exhaust,loop = 0,0,0,0,0
                total_yes=0
                total_yes_decs=0
                rep = 0
                while outcome ==0 and exhaust ==0: #while not engage or exhaust
                    decision = one_decision_CBT(mn,sk, dep,en,loop*0.2)
                    outcome = decision[0]
                    en+=decision[1]
                    total_yes+=decision[2]
                    total_yes_decs+=decision[3]
                    rep+=decision[4]
                    exhaust = decision[5]
                    loop+=1
                predicted = 0
                if total_yes_decs!=0:
                    predicted = total_yes/total_yes_decs
                want = 0
                if decision[6]!=0:
                    want = predicted

                if outcome==1 and flg ==1:
                    flg_con = 1 # two engages in a row
                else:
                    flg_con = 0
                flg = outcome #1 for engage
                day_en+=en #energy used by decision
                rep = rep # deliberations of a decisions
                dec+=1
                en_flg+=exhaust  #1 for exhaustion
                if flg ==1:
                    yes+=1
                elif flg==0 and exhaust ==0:
                    no+=1

                learn = param_learn(mood,m1,predicted,flg,flg_con,sk,d,day_actual,actual_pos,dep,exhaust,want)

            elif  disengage ==0: #not forcing disengage and no CBT
                decision = one_decision(mn,sk, dep)
                if decision[0]==1 and flg ==1:
                    flg_con = 1 # two engages in a row
                else:
                    flg_con = 0

                flg = decision[0] #1 for engage
                en_flg+=decision[4]  #1 for exhaustion
                day_en+=decision[1] #energy used by decision
                rep=decision[3] # deliberations of a decisions
                dec+=1

                if flg ==1:
                    yes+=1
                elif flg==0 and decision[4] ==0:
                    no+=1
                
                learn = param_learn(mood,m1,decision[2],flg,flg_con,sk,d,day_actual,actual_pos,dep,decision[4],decision[5]) # learning process to update m1 (hence mean) skew and apathy
            
            else: #forcing disengage
                
                flg_con = 0
                flg = 0
                rep = 0
                dec+=1
                no+=1
                learn = param_learn(mood,m1,0,flg,flg_con,sk,d,day_actual,actual_pos,dep,0,0)
                
            m1 = learn[0]
            
            sk+=learn[1]
            ap+=learn[2]
            mood+=learn[3]/10 # divide by 10 for stable behaviour
            
            if mood<0:
                mood = 0
            if mood > md_cap: #Cap is 1 when we dont force mood
                mood = md_cap
            
            if ap<0:
                ap = 0
            if ap>10:
                ap = 10
            
            
            ev_rep.append(rep)
            
            
            day_en_th = day_en_th_fun(dep) #update daily en_th for new ap
           
        sk+=learn_sk(yes,sk) #update the skew after each day with the number of engages made in a day

        if day_en>=day_en_th: #Ran out of energy in a day
            rep = 0
            flg = 0
            flg_con = 0
            day_exhaust=10-dec

            for i in range(10-dec):
                #Updating still occurs during complete exhaustion, just the updating of not engaging
                mn = mean((1-mood),m1)

                learn = param_learn(mood,m1,0,flg,flg_con,sk,d,day_actual,actual_pos,dep,1,0) # learning process to update m1 (hence mean) skew and apathy
                m1 = learn[0]
                sk+=learn[1]
                ap+=learn[2]
                
                if ap<0:
                    ap = 0
                if ap>10:
                    ap = 10
               
                mood+=learn[3]/10

                if mood > md_cap: #Cap is 1 when we dont force mood
                    mood = md_cap
                if mood<0:
                    mood = 0

                en_flg+=1 #still counts as a decision made by exhaustion
                
                ev_rep.append(rep)
                

        dep = ap/10*0.3+(1-mood)*(1-0.3)  #Dep depends more on low mood than apathy

        ev_mn.append(mn)
        ev_sk.append(sk)
        ev_ap.append(ap)
        ev_pos_dec.append(yes)
        ev_neg_dec.append(no)
        ev_exhaust.append(en_flg)
        ev_mood.append(mood)
        ev_dep.append(dep)
        ev_day_exhaust.append(day_exhaust)
        
    return [ev_mn,ev_sk,ev_ap,ev_rep,ev_pos_dec,ev_neg_dec, ev_exhaust,ev_day_exhaust,ev_mood,ev_dep]


In [89]:
def plots(dep, days,f_mn=None,f_sk=None,f_ap=None,f_ac=None,f_en=None,f_CBTc=None,f_ILM=None,f_ID=None, f_CBTb=None,save = False,raw=False):
    plt.rcParams["figure.figsize"] = (6, 3) #width,height # I believe this will get divided by subplots, make sure ratios are constant across plots
    plt.rcParams.update({'font.size':11})
    plt.rcParams["font.family"] = "Times New Roman"
    
    days  = math.ceil(days/14)*14 #14 day time averaging

    x = np.arange(0,days,14)

    out = day_decisions(dep , days, f_mn,f_sk,f_ap,f_ac,f_en,f_CBTc,f_ILM,f_ID, f_CBTb)
    

    y1  = np.asarray(out[0]) #mean
    y1 = np.mean(y1.reshape(-1, 14), axis=1)
    y2  = np.asarray(out[1]) #skew
    y2 = np.mean(y2.reshape(-1, 14), axis=1)
    y3  = np.asarray(out[2]) #apathy
    y3 = np.mean(y3.reshape(-1, 14), axis=1)
    y4  = np.asarray(out[3]) #deliberations, per decision
  
    y5  = np.asarray(out[4]) #pos_dec
    y5 = np.mean(y5.reshape(-1, 14), axis=1)
    y6  = np.asarray(out[5]) #neg_dec
    y6 = np.mean(y6.reshape(-1, 14), axis=1)
    y7  = np.asarray(out[6]) #exhausts
    y7 = np.mean(y7.reshape(-1, 14), axis=1)
    y8  = np.asarray(out[7]) #day exhausts
    y8 = np.mean(y8.reshape(-1, 14), axis=1)
    y9  = np.asarray(out[8]) #mood
    y9 = np.mean(y9.reshape(-1, 14), axis=1)
    y10 = np.asarray(out[9]) #depression
    y10 = np.mean(y10.reshape(-1, 14), axis=1)

    
    #Cluster 1
    fig1, axs1 = plt.subplots(2,sharex=True)
    axs1[1].set_xlabel('Days')
    #axs1.ticklabel_format(useOffset=False)

    axs1[0].plot(x, y1) 
    axs1[0].set(ylabel = 'Mean')
    axs1[0].set_ylim([-0.45, 0.25]) 

    axs1[1].plot(x, y2)
    axs1[1].set(ylabel = 'Skew')
    #axs1[1].set_ylim([-0.25, 0.15])

    axs1[0].set_yticks([-0.4,-0.2,0,0.2])
    axs1[0].set_yticklabels([-0.4,-0.2,0,0.2])

    #axs1[1].set_ylim(skew_y_lim)                                 ###########################################
    #axs1[1].set_yticks(skew_yticks)                                  ###########################################
    #axs1[1].set_yticklabels(skew_yticks)                         ###########################################
    plt.subplots_adjust(hspace=0)
    if save == False:
        plt.show()
    else:
        plt.savefig('Plots/'+ save +'_distribution',bbox_inches = 'tight')
        plt.savefig('Plots/'+ save +'_distribution.pdf',bbox_inches = 'tight')

    #Cluster 2
    fig2, axs2 = plt.subplots(2,gridspec_kw={'height_ratios': [3, 1]},sharex=True)
    axs2[1].set_xlabel('Days')
    #axs2.ticklabel_format(useOffset=False)
    axs2[0].set_ylim([-1, 11])
    
   
    axs2[0].plot(x, y5, label = 'Engages')
    axs2[0].plot(x, y6, label = 'Active Disengages')
    axs2[0].plot(x, y7, label = 'Exhaustion Disengages')
    axs2[0].legend(prop={'size':9})
    axs2[0].set(ylabel = 'Decisions')
    axs2[0].set_yticks([0,5,10])
    axs2[0].set_yticklabels([0,5,10])

    axs2[1].bar(x,y8, width = 13)
    axs2[1].set(ylabel = 'Fatigue \n Disengages') 
    #axs2[1].set_yticks([0,1])                                       ###########################################
    #axs2[1].set_yticklabels([0,1])                                   ###########################################
    #axs2[1].set_ylim([-0.1 ,fatigue_max+0.2])                           #############################################

    plt.subplots_adjust(hspace=0)
    
    if save == False:
        plt.show()
    else:
        plt.savefig('Plots/'+ save +'_engages',bbox_inches = 'tight')
        plt.savefig('Plots/'+ save +'_engages.pdf',bbox_inches = 'tight')

    #Cluster 3
    fig3, axs3 = plt.subplots(3,sharex=True)
    axs3[2].set_xlabel('Days')
    #axs3.ticklabel_format(useOffset=False)
    axs3[0].set_ylim([-1, 11])
    axs3[1].set_ylim([-0.1, 1.1])
    axs3[2].set_ylim([-0.1, 1.1])

    axs3[0].plot(x, y3)
    axs3[0].set(ylabel ='Apathy') #Plot and amybe zoom in on a point to show small flucuations
    axs3[1].plot(x, y9)
    axs3[1].set(ylabel = 'Mood') # plot with MDD and Ap
    axs3[2].plot(x, y10)
    axs3[2].set(ylabel = 'MDD') #Could plot maximum and Minimum Value due to apathy 

    axs3[0].set_yticks([0,5,10])
    axs3[0].set_yticklabels([0,0.5,1])
    axs3[1].set_yticks([0,0.5,1])
    axs3[1].set_yticklabels([0,0.5,1])
    axs3[2].set_yticks([0,0.5,1])
    axs3[2].set_yticklabels([0,0.5,1])
    
    
    if save == False:
        plt.show()
    else:
        plt.savefig('Plots/'+ save +'_MDD',bbox_inches = 'tight')
        plt.savefig('Plots/'+ save +'_MDD.pdf',bbox_inches = 'tight')

    if raw ==True:
        return [y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,['mean','skew','apathy','deliberation','engages','disengage','exhaustion','fatigues','mood','depression']]
    
    
    

In [ ]:
#A standard run will last for 500+ days, with an inital depression level = 0
#ILM lasting 30 days with a value of 0.2 and ID lasting ID is likely to cause an episode
#CBTb and CBTc lasting 400 days is a standard treatment length